# Setup OpenEBS

In [94]:
!aws ec2 describe-instances \
        --query 'Reservations[].Instances[].[Tags[?Key==`Name`].Value[] | [0], InstanceId, Placement.AvailabilityZone]' \
        --output text

cluster-cpu-nodes-Node	i-0bf31bc06d431a3b4	us-west-2a
cluster-cpu-nodes-Node	i-0e1e3cc267f54e715	us-west-2c
cluster-cpu-nodes-Node	i-0817a7d327859fae3	us-west-2c
cluster-cpu-nodes-Node	i-08612759f8e75d6e4	us-west-2d


In [96]:
!aws ec2 describe-instances \
    --query 'Reservations[*].Instances[*].[Tags[?Key==`Name`].Value,InstanceId,BlockDeviceMappings[*].Ebs.VolumeId]' \
    --output text

i-0bf31bc06d431a3b4
cluster-cpu-nodes-Node
vol-09722e1b454447a05
i-0e1e3cc267f54e715
cluster-cpu-nodes-Node
vol-007750725871cd26e	vol-04a1b2bd8346671c1	vol-0f76f1aee5701e5a6
i-0817a7d327859fae3
cluster-cpu-nodes-Node
vol-09c3a0c18bae7b896	vol-06ee4851914ec0a6f
i-08612759f8e75d6e4
cluster-cpu-nodes-Node
vol-06496a8c33dbdd9a8	vol-0aa9c83fe4f5b4c8e	vol-0fe83c47252fb76b3


# Install OpenEBS
https://docs.openebs.io/docs/next/uglocalpv-hostpath.html#create-storageclass

In [ ]:
# DEFAULT: The absolute path on the node where the Hostpath directory of a Local PV Volume will be created.
OPENEBS_IO_LOCALPV_HOSTPATH_DIR='/var/openebs/local'

In [27]:
!wget https://openebs.github.io/charts/openebs-operator.yaml

--2020-10-29 10:10:17--  https://openebs.github.io/charts/openebs-operator.yaml
Resolving openebs.github.io (openebs.github.io)... 185.199.110.153, 185.199.108.153, 185.199.109.153, ...
Connecting to openebs.github.io (openebs.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28070 (27K) [text/yaml]
Saving to: ‘openebs-operator.yaml.1’

openebs-operator.ya 100%[===================>]  27.41K  --.-KB/s    in 0.009s  

2020-10-29 10:10:17 (3.14 MB/s) - ‘openebs-operator.yaml.1’ saved [28070/28070]



In [28]:
#!pygmentize openebs-operator.yaml

In [29]:
!kubectl apply -f https://openebs.github.io/charts/openebs-operator.yaml

namespace/openebs created
serviceaccount/openebs-maya-operator created
clusterrole.rbac.authorization.k8s.io/openebs-maya-operator created
clusterrolebinding.rbac.authorization.k8s.io/openebs-maya-operator created
deployment.apps/maya-apiserver created
service/maya-apiserver-service created
deployment.apps/openebs-provisioner created
deployment.apps/openebs-snapshot-operator created
configmap/openebs-ndm-config created
daemonset.apps/openebs-ndm created
deployment.apps/openebs-ndm-operator created
deployment.apps/openebs-admission-server created
deployment.apps/openebs-localpv-provisioner created


In [30]:
!kubectl get pods -n openebs

NAME                                           READY   STATUS    RESTARTS   AGE
maya-apiserver-649b4db7d4-xfztk                0/1     Running   0          8s
openebs-admission-server-5cff586c89-254c2      1/1     Running   0          8s
openebs-localpv-provisioner-6498bddcff-bjdlm   1/1     Running   0          8s
openebs-ndm-chn4f                              1/1     Running   0          8s
openebs-ndm-kfv2h                              1/1     Running   0          7s
openebs-ndm-operator-68bc9775cb-r2gf7          0/1     Running   0          8s
openebs-ndm-rbfqd                              1/1     Running   0          8s
openebs-ndm-sxqfk                              1/1     Running   0          8s
openebs-provisioner-69489fb565-v6fkz           1/1     Running   0          8s
openebs-snapshot-operator-5c758ff48f-cj859     2/2     Running   0          8s


# Create StorageClass

In [48]:
!pygmentize eks-openebs/local-hostpath-sc.yaml

apiVersion: storage.k8s.io/v1
kind: StorageClass
metadata:
  name: local-hostpath
  annotations:
    openebs.io/cas-type: local
    cas.openebs.io/config: |
      - name: StorageType
        value: hostpath
      - name: BasePath
        value: /opt/ml
provisioner: openebs.io/local
reclaimPolicy: Delete
volumeBindingMode: WaitForFirstConsumer


In [14]:
#!kubectl delete sc local-hostpath

In [49]:
!kubectl create -f local-hostpath-sc.yaml

storageclass.storage.k8s.io/local-hostpath created


In [50]:
!kubectl get sc local-hostpath -o yaml

apiVersion: storage.k8s.io/v1
kind: StorageClass
metadata:
  annotations:
    cas.openebs.io/config: |
      - name: StorageType
        value: hostpath
      - name: BasePath
        value: /opt/ml
    openebs.io/cas-type: local
  creationTimestamp: "2020-10-29T10:28:34Z"
  name: local-hostpath
  resourceVersion: "28589143"
  selfLink: /apis/storage.k8s.io/v1/storageclasses/local-hostpath
  uid: 199b866f-a1cd-4d7d-9c70-bb79e7eb907d
provisioner: openebs.io/local
reclaimPolicy: Delete
volumeBindingMode: WaitForFirstConsumer


## Verify 

In [51]:
!kubectl get pods -n openebs -l openebs.io/component-name=openebs-localpv-provisioner

NAME                                           READY   STATUS    RESTARTS   AGE
openebs-localpv-provisioner-6498bddcff-bjdlm   1/1     Running   0          18m


In [52]:
!kubectl get sc

NAME                        PROVISIONER                                                RECLAIMPOLICY   VOLUMEBINDINGMODE      ALLOWVOLUMEEXPANSION   AGE
gp2 (default)               kubernetes.io/aws-ebs                                      Delete          WaitForFirstConsumer   false                  53d
local-hostpath              openebs.io/local                                           Delete          WaitForFirstConsumer   false                  43s
openebs-device              openebs.io/local                                           Delete          WaitForFirstConsumer   false                  30m
openebs-hostpath            openebs.io/local                                           Delete          WaitForFirstConsumer   false                  18m
openebs-jiva-default        openebs.io/provisioner-iscsi                               Delete          Immediate              false                  30m
openebs-snapshot-promoter   volumesnapshot.external-storage.k8s.io/snapshot-promot

# Create a PersistentVolumeClaim

In [53]:
!pygmentize eks-openebs/local-hostpath-pvc.yaml

kind: PersistentVolumeClaim
apiVersion: v1
metadata:
  name: local-hostpath-pvc
spec:
  storageClassName: local-hostpath
  accessModes:
    - ReadWriteOnce
  resources:
    requests:
      storage: 5G


In [54]:
!kubectl apply -f eks-openebs/local-hostpath-pvc.yaml

persistentvolumeclaim/local-hostpath-pvc created


In [55]:
!kubectl get pvc local-hostpath-pvc

NAME                 STATUS    VOLUME   CAPACITY   ACCESS MODES   STORAGECLASS     AGE
local-hostpath-pvc   Pending                                      local-hostpath   4s


# Create Pod to consume OpenEBS Local PV Hospath Storage

In [112]:
!kubectl get pods

NAME                                                           READY   STATUS      RESTARTS   AGE
admission-webhook-bootstrap-stateful-set-0                     1/1     Running     6          53d
admission-webhook-deployment-59bc556b94-x544t                  1/1     Running     0          13h
alb-ingress-controller-798fbd8c96-b6bh4                        1/1     Running     2          53d
application-controller-stateful-set-0                          1/1     Running     0          53d
argo-ui-5f845464d7-k28qc                                       1/1     Running     0          53d
bert-ml-pod                                                    0/1     Error       0          175m
bert-ml-pod-eks                                                0/1     Completed   0          98m
centraldashboard-d5c6d6bf-5scrq                                1/1     Running     0          53d
jupyter-web-app-deployment-544b7d5684-6r7g4                    1/1     Running     0          53d
katib-controller-6b

In [91]:
!kubectl get pods bert-ml-pod-openebs

Error from server (NotFound): pods "bert" not found


In [89]:
!kubectl delete pod bert-ml-pod-openebs

pod "bert" deleted


In [92]:
!kubectl apply -f eks-openebs/bert-ml-pod-openebs.yaml

pod/bert-ml-pod created


In [87]:
!kubectl logs -p bert-ml-pod-openebs

Error from server (BadRequest): previous terminated container "bert" in pod "bert" not found


In [88]:
!kubectl describe pod bert-ml-pod-openebs

Name:         bert
Namespace:    kubeflow
Priority:     0
Node:         ip-192-168-67-206.us-west-2.compute.internal/192.168.67.206
Start Time:   Thu, 29 Oct 2020 10:40:38 +0000
Labels:       <none>
Annotations:  kubectl.kubernetes.io/last-applied-configuration:
                {"apiVersion":"v1","kind":"Pod","metadata":{"annotations":{},"name":"bert","namespace":"kubeflow"},"spec":{"containers":[{"command":["pyth...
              kubernetes.io/psp: eks.privileged
Status:       Failed
IP:           192.168.68.249
Containers:
  bert:
    Container ID:  docker://5f3fd1e1ee40b54dd8ded73d9f62fe7cf33c22e2241d2a2291b209526257487e
    Image:         231218423789.dkr.ecr.us-west-2.amazonaws.com/dlc-demo:bert
    Image ID:      docker-pullable://231218423789.dkr.ecr.us-west-2.amazonaws.com/dlc-demo@sha256:4bf49dbd7bdb5ab8a4c76086622ccfaa893690047c744d68a93d9ee577f1720f
    Port:          <none>
    Host Port:     <none>
    Command:
      python
      /opt/ml/code/train.py
      --train_steps_p

In [57]:
#!pygmentize eks-openebs/bert-ml-pod-openebs.yaml

In [76]:
!kubectl delete -f eks-openebs/bert-ml-pod-openebs.yaml

pod "bert" deleted


In [93]:
!kubectl get pod bert-ml-pod-openebs

NAME          READY   STATUS    RESTARTS   AGE
bert-ml-pod   1/1     Running   0          9s
Error from server (NotFound): pods "created" not found


In [74]:
!kubectl describe pod bert-ml-pod-openebs

Name:         bert
Namespace:    kubeflow
Priority:     0
Node:         ip-192-168-67-206.us-west-2.compute.internal/192.168.67.206
Start Time:   Thu, 29 Oct 2020 10:36:59 +0000
Labels:       <none>
Annotations:  kubectl.kubernetes.io/last-applied-configuration:
                {"apiVersion":"v1","kind":"Pod","metadata":{"annotations":{},"name":"bert","namespace":"kubeflow"},"spec":{"containers":[{"command":["pyth...
              kubernetes.io/psp: eks.privileged
Status:       Failed
IP:           192.168.68.249
Containers:
  bert:
    Container ID:  docker://1fdf43c6b3513c8ab6fa3b3b784611808c687eb0b4356af9661f8ac05c5d807f
    Image:         231218423789.dkr.ecr.us-west-2.amazonaws.com/dlc-demo:bert
    Image ID:      docker-pullable://231218423789.dkr.ecr.us-west-2.amazonaws.com/dlc-demo@sha256:4bf49dbd7bdb5ab8a4c76086622ccfaa893690047c744d68a93d9ee577f1720f
    Port:          <none>
    Host Port:     <none>
    Command:
      python
      /opt/ml/code/train.py
      --train_steps_p

In [63]:
!kubectl get pvc local-hostpath-pvc

NAME                 STATUS    VOLUME   CAPACITY   ACCESS MODES   STORAGECLASS     AGE
local-hostpath-pvc   Pending                                      local-hostpath   4m57s


In [ ]:
!kubectl get pv pvc-864a5ac8-dd3f-416b-9f4b-ffd7d285b425 -o yaml

# Delete

In [41]:
#!kubectl delete pod hello-local-hostpath-pod

In [45]:
#!kubectl delete pvc local-hostpath-pvc

In [47]:
#!kubectl delete sc local-hostpath

In [ ]:
#!kubectl delete -f https://openebs.github.io/charts/openebs-operator.yaml